In [16]:
import tensorflow as tf
import datetime

In [17]:
# make sure TensorFlow can see the GPU
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [4]:
mnist = tf.keras.datasets.mnist

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28)), 
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [9]:
model = create_model()
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [11]:
log_dir = "logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [31]:
model.fit(x=x_train, 
          y=y_train, 
          epochs=10, 
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_cb]
         )

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0132 - accuracy: 0.9955 - val_loss: 0.0862 - val_accuracy: 0.9834
Epoch 2/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0133 - accuracy: 0.9959 - val_loss: 0.0910 - val_accuracy: 0.9842
Epoch 3/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0122 - accuracy: 0.9961 - val_loss: 0.0958 - val_accuracy: 0.9846
Epoch 4/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0127 - accuracy: 0.9962 - val_loss: 0.0897 - val_accuracy: 0.9848
Epoch 5/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0105 - accuracy: 0.9966 - val_loss: 0.0988 - val_accuracy: 0.9839
Epoch 6/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0116 - accuracy: 0.9963 - val_loss: 0.1015 - val_accuracy: 0.9845
Epoch 7/10
60000/60000 [==============================] 